In [1]:
from hydra import compose, initialize

from dataset import get_transforms, load_datalist

with initialize(version_base=None, config_path="config", job_name="test_app"):
    cfg = compose(config_name="root", overrides=['data=ntuh', 'debug=true'])

datalist = load_datalist(cfg)

In [2]:
from utils import get_class
from monai.data import ThreadDataLoader

dataset_class = get_class(cfg.dataset.type)
datasets = {
    k: dataset_class(data=v if not cfg.debug else v[:5],
                     transform=get_transforms(k, cfg), **cfg.dataset.params)
    for k, v in datalist.items()
}

dataloaders = {k: ThreadDataLoader(v, batch_size=cfg.batch_size[k] if k == 'train' else 1,
                                   use_thread_workers=True, buffer_size=cfg.buffer_size,
                                   num_workers=cfg.num_workers)
               for k, v in datasets.items()}

Loading dataset: 100%|██████████| 5/5 [00:04<00:00,  1.04it/s]


In [4]:
import torch

for batch in dataloaders['train']:
    print(batch['image'].shape, batch['label'].shape)
    break

torch.Size([2, 1, 192, 192, 64]) torch.Size([2, 1, 192, 192, 64])


In [4]:
from monai.transforms import AsDiscrete

discretize = AsDiscrete(to_onehot=5)

In [10]:
import torch

x = torch.randint(0, 10, (1, 3, 3))

x

tensor([[[7, 6, 5],
         [9, 5, 8],
         [9, 0, 3]]])

In [5]:
datalist

{'train': [{'image': 'D:\\Dropbox\\Datasets\\HepaticVessel\\images\\hepaticvessel_012.nii.gz',
   'label': 'D:\\Dropbox\\Datasets\\HepaticVessel\\labels\\hepaticvessel_012.seg.nrrd'},
  {'image': 'D:\\Dropbox\\Datasets\\HepaticVessel\\images\\hepaticvessel_013.nii.gz',
   'label': 'D:\\Dropbox\\Datasets\\HepaticVessel\\labels\\hepaticvessel_013.seg.nrrd'},
  {'image': 'D:\\Dropbox\\Datasets\\HepaticVessel\\images\\hepaticvessel_014.nii.gz',
   'label': 'D:\\Dropbox\\Datasets\\HepaticVessel\\labels\\hepaticvessel_014.seg.nrrd'},
  {'image': 'D:\\Dropbox\\Datasets\\HepaticVessel\\images\\hepaticvessel_015.nii.gz',
   'label': 'D:\\Dropbox\\Datasets\\HepaticVessel\\labels\\hepaticvessel_015.seg.nrrd'},
  {'image': 'D:\\Dropbox\\Datasets\\HepaticVessel\\images\\hepaticvessel_016.nii.gz',
   'label': 'D:\\Dropbox\\Datasets\\HepaticVessel\\labels\\hepaticvessel_016.seg.nrrd'},
  {'image': 'D:\\Dropbox\\Datasets\\HepaticVessel\\images\\hepaticvessel_017.nii.gz',
   'label': 'D:\\Dropbox\\Data